In [1]:
import pyreadr
import os 
import pandas as pd
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

df_sample = list(pyreadr.read_r('data/db_hex.rds').values())[0]


c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification\Py\notebooks


NameError: name 'df' is not defined

In [4]:
df = df_sample.sample(n=10000)

In [5]:
import re

# Spalten "titel" und "kursbeschreibung" zusammenführen mit ": " als Trennzeichen
df['sentence'] = df['titel'] + ": " + df['kursbeschreibung']

# Entfernen der alphanumerischen Codes mit Punkten und Bindestrichen
df['sentence'] = df['sentence'].str.replace(r'\b[\w]+([.-][\w]+)+\b', '', regex=True)

 
# Entfernen von unnötigen Leerzeichen
df['sentence'] = df['sentence'].str.strip().replace(r'\s+', ' ', regex=True)

In [6]:
import torch
torch.cuda.empty_cache()

In [7]:
from tqdm import tqdm
tqdm.pandas()

In [14]:
import pandas as pd
from setfit import SetFitModel

# Laden des vortrainierten Modells von Hugging Face
model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")
model = SetFitModel.from_pretrained("models")  # Ändere den Pfad zu deinem Modell


# Definiere die Labels in der richtigen Reihenfolge
fs_labels = ['Hardware-/Robotikentwicklung', 'Softwareentwicklung', 'Nutzerzentriertes Design', 
             'Data Analytics & KI', 'Quantencomputing', 'IT-Architektur']

# Definiere eine Funktion, die das Modell auf eine Kursbeschreibung anwendet
def predict_course_description(description):
    # Überprüfen, ob die Beschreibung ein String ist
    if isinstance(description, str):
        preds = model(description)
        return preds
    # Falls die Beschreibung keine gültige Zeichenfolge ist, gib eine leere Liste oder eine andere Standardrückgabe zurück
    return []

# Funktion, um Vorhersagen in String-Variable umzuwandeln
def convert_tensor_to_labels(tensor):
    # Identifiziere die Positionen, wo der Wert 1 ist
    selected_labels = [fs_labels[i] for i, val in enumerate(tensor) if val == 1]
    # Falls keine Labels ausgewählt wurden, gib "Keine" zurück
    return ', '.join(selected_labels) if selected_labels else 'Keine'

# Wende die Vorhersagen und die Umwandlungsfunktion auf jede Kursbeschreibung an
df["Vorhersagen"] = df["sentence"].progress_apply(predict_course_description)
df["FS_Skill"] = df["Vorhersagen"].progress_apply(lambda x: convert_tensor_to_labels(x))



# DataFrame mit den neuen FS_Skill-Spalten anzeigen
print(df)


100%|██████████| 10000/10000 [00:00<00:00, 54650.05it/s]

            hochschulname_kurz  \
49081                   U Köln   
1254670              TU Berlin   
565305                U Bochum   
1426701               CAU Kiel   
1871053               U Bremen   
...                        ...   
961777   FAU Erlangen-Nürnberg   
257648             U Frankfurt   
1116617              HU Berlin   
154942              TU München   
1568442               CAU Kiel   

                                                hochschule  \
49081                                  Universität zu Köln   
1254670                      Technische Universität Berlin   
565305                             Ruhr-Universität Bochum   
1426701            Christian-Albrechts-Universität zu Kiel   
1871053                                 Universität Bremen   
...                                                    ...   
961777   Friedrich-Alexander-Universität Erlangen-Nürnberg   
257648   Johann Wolfgang Goethe-Universität Frankfurt a...   
1116617                     Humbo

In [9]:
type(df["Vorhersagen"])

pandas.core.series.Series

In [16]:
# Exportiere den DataFrame in eine Excel-Datei
# Exportiere den DataFrame in eine Excel-Datei
df.to_csv('output_large.csv', index=False)





In [15]:
label_counts = df['FS_Skill'].value_counts()